In [1]:
import os
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.v1.extensions.google_cloud_big_query import BigQueryExampleGen
from tfx.v1.components import StatisticsGen, SchemaGen, Transform

PROJECT_ID = "uci-bank-marketing-ml-lab"
GCP_REGION = "us-central1"

BQ_PROJECT = PROJECT_ID
BQ_DATASET = "bank_data"
BQ_TABLE = "uci_bank_marketing"

PIPELINE_NAME = "bq_classifier_validation"
LOCAL_PIPELINE_ROOT = os.path.join(os.getcwd(), "tfx_local", PIPELINE_NAME)
os.makedirs(LOCAL_PIPELINE_ROOT, exist_ok=True)

os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID
os.environ["PROJECT_ID"] = PROJECT_ID
os.environ["GCLOUD_PROJECT"] = PROJECT_ID

context = InteractiveContext(pipeline_root=LOCAL_PIPELINE_ROOT)
print("Using pipeline_root:", context.pipeline_root)

TMP_LOCATION = "gs://uci-bank-ml-bucket/tfx/tmp"

beam_pipeline_args = [
    f"--project={PROJECT_ID}",
    f"--region={GCP_REGION}",
    f"--temp_location={TMP_LOCATION}",
]


2025-11-11 02:09:16.430818: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-11 02:09:16.437189: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-11 02:09:16.452686: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-11-11 02:09:16.482417: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-11-11 02:09:16.482467: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-11-11 02:09:16.502400: I tensorflow/core/platform/cpu_feature_guard.cc:

Using pipeline_root: /home/jupyter/tfx_local/bq_classifier_validation


/home/jupyter/tfx-env/lib/python3.10/site-packages/google/cloud/aiplatform/models.py:52: FutureWarning: Support for google-cloud-storage < 3.0.0 will be removed in a future version of google-cloud-aiplatform. Please upgrade to google-cloud-storage >= 3.0.0.
  from google.cloud.aiplatform.utils import gcs_utils


### ExampleGen

In [2]:
query = f"""
SELECT *
FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}`
"""

example_gen = BigQueryExampleGen(query=query)
context.run(example_gen, beam_pipeline_args=beam_pipeline_args)


ExecutionResult(
    component_id: BigQueryExampleGen
    execution_id: 14
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=BigQueryExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

### StatisticsGen

In [3]:
statistics_gen = StatisticsGen(examples=example_gen.outputs["examples"])
context.run(statistics_gen, beam_pipeline_args=beam_pipeline_args)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 15
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

### SchemaGen

In [4]:
schema_gen = SchemaGen(
    statistics=statistics_gen.outputs["statistics"],
    infer_feature_shape=True,
)
context.run(schema_gen, beam_pipeline_args=beam_pipeline_args)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 16
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [5]:
context.show(schema_gen.outputs["schema"])

,Type,Presence,Valency,Domain
Feature name,,,,
'age',INT,required,,-
'campaign',INT,required,,-
'cons_conf_idx',FLOAT,required,,-
'cons_price_idx',FLOAT,required,,-
'contact',STRING,required,,'contact'
'day_of_week',STRING,required,,'day_of_week'
'default',STRING,required,,'default'
'duration',INT,required,,-
'education',STRING,required,,'education'


,Values
Domain,
'contact',"'cellular', 'telephone'"
'day_of_week',"'fri', 'mon', 'thu', 'tue', 'wed'"
'default',"'no', 'unknown', 'yes'"
'education',"'basic.4y', 'basic.6y', 'basic.9y', 'high.school', 'illiterate', 'professional.course', 'university.degree', 'unknown'"
'housing',"'no', 'unknown', 'yes'"
'job',"'admin.', 'blue-collar', 'entrepreneur', 'housemaid', 'management', 'retired', 'self-employed', 'services', 'student', 'technician', 'unemployed', 'unknown'"
'loan',"'no', 'unknown', 'yes'"
'marital',"'divorced', 'married', 'single', 'unknown'"
'month',"'apr', 'aug', 'dec', 'jul', 'jun', 'mar', 'may', 'nov', 'oct', 'sep'"


In [9]:
%%writefile transform_module.py
import tensorflow as tf
import tensorflow_transform as tft

NUMERIC_FEATURES = [
    "age",
    "campaign",
    "pdays",
    "previous",
    "emp_var_rate",
    "cons_price_idx",
    "cons_conf_idx",
    "euribor3m",
    "nr_employed",
    "duration",
]

CATEGORICAL_FEATURES = [
    "job",
    "marital",
    "education",
    "default",
    "housing",
    "loan",
    "contact",
    "month",
    "day_of_week",
    "poutcome",
]

LABEL_KEY = "y"  


def _make_label(x):
    """Handle both string and numeric label representations."""
    if x.dtype == tf.string:
        # UCI style: "yes" / "no"
        return tf.cast(tf.equal(x, b"yes"), tf.int64)
    else:
        # Already numeric (0/1 or similar) -> just cast
        return tf.cast(x, tf.int64)


def preprocessing_fn(inputs):
    outputs = {}

    # Numeric features -> z-score normalized
    for key in NUMERIC_FEATURES:
        if key in inputs:
            outputs[key + "_z"] = tft.scale_to_z_score(
                tf.cast(inputs[key], tf.float32)
            )

    # Categorical features -> vocab IDs
    for key in CATEGORICAL_FEATURES:
        if key in inputs:
            outputs[key + "_id"] = tft.compute_and_apply_vocabulary(
                inputs[key],
                top_k=100,
                num_oov_buckets=1,
            )

    # Label -> 0/1 int64
    if LABEL_KEY in inputs:
        outputs[LABEL_KEY] = _make_label(inputs[LABEL_KEY])

    return outputs


Overwriting transform_module.py


In [10]:
from tfx.v1.components import Transform
import os

transform = Transform(
    examples=example_gen.outputs["examples"],
    schema=schema_gen.outputs["schema"],
    module_file=os.path.join(os.getcwd(), "transform_module.py"),
)

context.run(transform, beam_pipeline_args=beam_pipeline_args)


running bdist_wheel
running build
running build_py
creating build/lib
copying transform_module.py -> build/lib
installing to /var/tmp/tmpfa332s_t
running install
running install_lib
copying build/lib/transform_module.py -> /var/tmp/tmpfa332s_t/.
running install_egg_info
running egg_info
creating tfx_user_code_Transform.egg-info
writing tfx_user_code_Transform.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Transform.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Transform.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
Copying tfx_user_code_Transform.egg-info to /var/tmp/tmpfa332s_t/./tfx_user_code_Transform-0.0+5daa7850faa16129860ef49256913be91dfc801060b2fc537e7a508db9e202b3-py3.10.egg-info
running install_scripts
creating /var/tmp/tmpfa332s_t/tfx_user_code_transform-0.0

/home/jupyter/tfx-env/lib/python3.10/site-packages/setuptools/_distutils/cmd.py:90: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        This deprecation is overdue, please update your project and remove deprecated
        calls to avoid build errors in the future.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()


Processing ./tfx_local/bq_classifier_validation/_wheels/tfx_user_code_transform-0.0+5daa7850faa16129860ef49256913be91dfc801060b2fc537e7a508db9e202b3-py3-none-any.whl
Processing ./tfx_local/bq_classifier_validation/_wheels/tfx_user_code_transform-0.0+5daa7850faa16129860ef49256913be91dfc801060b2fc537e7a508db9e202b3-py3-none-any.whl
Processing ./tfx_local/bq_classifier_validation/_wheels/tfx_user_code_transform-0.0+5daa7850faa16129860ef49256913be91dfc801060b2fc537e7a508db9e202b3-py3-none-any.whl


ExecutionResult(
    component_id: Transform
    execution_id: 18
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [11]:
transform.outputs

{'transform_graph': OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False),
 'transformed_examples': OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False),
 'updated_analyzer_cache': OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False),
 'pre_transform_schema': OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False),
 'pre_transform_stats': OutputChannel(artifact_type=ExampleStatistics, producer_componen